In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, GlobalMaxPooling1D, Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [ ]:
%%writefile job_data.json
 {
  "job_fields": [
    {
      "field_name": "Information Technology (IT) and Computer Science",
      "job_examples": [
        {
          "job_title": "Software Developer",
          "work_field_scope": "Designs, codes, tests, and maintains software programs.",
          "skills": [
            "Programming languages (e.g., Java, Python, C++)",
            "Problem-solving and algorithmic skills",
            "Knowledge of software development methodologies"
          ],
          "requirements": [
            "Bachelor's degree in Computer Science or related field",
            "Relevant work experience may be required"
          ]
        },
        {
          "job_title": "Network Administrator",
          "work_field_scope": "Manages and maintains an organization's computer networks.",
          "skills": [
            "Networking protocols and technologies",
            "Troubleshooting and problem-solving skills",
            "Security fundamentals"
          ],
          "requirements": [
            "Bachelor's degree in IT or related field",
            "Relevant certifications (e.g., CCNA) may be required"
          ]
        },
        {
          "job_title": "Data Scientist",
          "work_field_scope": "Analyzes and interprets complex data sets to inform business decision-making.",
          "skills": [
            "Statistics and data analysis skills",
            "Programming languages (e.g., Python, R)",
            "Machine learning and data visualization expertise"
          ],
          "requirements": [
            "Master's degree in Data Science or related field",
            "Experience with data analysis tools and frameworks"
          ]
        }
      ]
    },
    {
      "field_name": "Healthcare",
      "job_examples": [
        {
          "job_title": "Registered Nurse",
          "work_field_scope": "Provides patient care, administers medications, and coordinates treatment plans.",
          "skills": [
            "Nursing degree and licensure",
            "Compassion and interpersonal skills",
            "Critical thinking in fast-paced environments"
          ],
          "requirements": [
            "Bachelor's degree in Nursing",
            "Clinical experience and licensing exam required"
          ]
        },
        {
          "job_title": "Medical Laboratory Technologist",
          "work_field_scope": "Performs tests on patient samples to assist in the diagnosis and treatment of diseases.",
          "skills": [
            "Bachelor's degree in medical technology",
            "Attention to detail and precision",
            "Knowledge of laboratory equipment and procedures"
          ],
          "requirements": [
            "Certification as a Medical Laboratory Technologist",
            "Clinical laboratory experience"
          ]
        },
        {
          "job_title": "Health Informatics Specialist",
          "work_field_scope": "Manages and analyzes healthcare data for improved patient outcomes.",
          "skills": [
            "Bachelor's degree in health informatics or related field",
            "Analytical and problem-solving skills",
            "Knowledge of healthcare information systems"
          ],
          "requirements": [
            "Relevant certifications (e.g., Certified Health Data Analyst)",
            "Experience in healthcare data analysis"
          ]
        }
      ]
    },
    {
      "field_name": "Finance",
      "job_examples": [
        {
          "job_title": "Financial Analyst",
          "work_field_scope": "Analyzes financial data to provide investment recommendations.",
          "skills": [
            "Analytical and quantitative skills",
            "Knowledge of financial modeling",
            "Understanding of economic trends"
          ],
          "requirements": [
            "Bachelor's degree in Finance, Economics, or related field",
            "Relevant certifications (e.g., CFA) may be required"
          ]
        },
        {
          "job_title": "Accountant",
          "work_field_scope": "Prepares and examines financial records to ensure accuracy and compliance.",
          "skills": [
            "Attention to detail and organizational skills",
            "Knowledge of accounting principles and regulations"
          ],
          "requirements": [
            "Bachelor's degree in Accounting or related field",
            "CPA certification may be required"
          ]
        },
        {
          "job_title": "Investment Banker",
          "work_field_scope": "Facilitates financial transactions and advises clients on investments.",
          "skills": [
            "Strong analytical and strategic thinking",
            "Financial modeling and valuation skills",
            "Excellent communication and negotiation skills"
          ],
          "requirements": [
            "Bachelor's or Master's degree in Finance, Business, or related field",
            "Relevant experience in investment banking"
          ]
        }
      ]
    },
    {
      "field_name": "Education",
      "job_examples": [
        {
          "job_title": "Teacher",
          "work_field_scope": "Educates students in a specific subject or grade level.",
          "skills": [
            "Bachelor's degree in Education",
            "Classroom management skills",
            "Effective communication and adaptability"
          ],
          "requirements": [
            "Teaching certification",
            "Student teaching experience"
          ]
        },
        {
          "job_title": "School Counselor",
          "work_field_scope": "Provides guidance and support to students.",
          "skills": [
            "Master's degree in Counseling or related field",
            "Empathy and interpersonal skills",
            "Knowledge of counseling techniques"
          ],
          "requirements": [
            "State licensure as a school counselor",
            "Relevant counseling experience"
          ]
        },
        {
          "job_title": "Education Administrator",
          "work_field_scope": "Manages the operations of educational institutions.",
          "skills": [
            "Master's degree in Educational Leadership or Administration",
            "Leadership and organizational skills",
            "Knowledge of education policies and regulations"
          ],
          "requirements": [
            "Administrative certification",
            "Experience in educational leadership"
          ]
        }
      ]
    },
    {
      "field_name": "Marketing and Advertising",
      "job_examples": [
        {
          "job_title": "Marketing Manager",
          "work_field_scope": "Develops marketing strategies and oversees campaigns.",
          "skills": [
            "Strategic planning and leadership skills",
            "Market research and analysis",
            "Communication and creativity"
          ],
          "requirements": [
            "Bachelor's or Master's degree in Marketing or related field",
            "Relevant experience in marketing"
          ]
        },
        {
          "job_title": "Digital Marketing Specialist",
          "work_field_scope": "Manages online marketing channels and campaigns.",
          "skills": [
            "SEO and SEM knowledge",
            "Social media marketing skills",
            "Data analysis and digital advertising expertise"
          ],
          "requirements": [
            "Bachelor's degree in Marketing, Digital Marketing, or related field",
            "Certifications in digital marketing (e.g., Google Ads) may be required"
          ]
        },
        {
          "job_title": "Advertising Copywriter",
          "work_field_scope": "Creates persuasive and engaging content for advertisements.",
          "skills": [
            "Creative writing and storytelling skills",
            "Understanding of brand messaging",
            "Collaboration with design and marketing teams"
          ],
          "requirements": [
            "Bachelor's degree in Advertising, Copywriting, or related field",
            "Portfolio showcasing copywriting skills"
          ]
        }
      ]
    },
    {
      "field_name": "Engineering",
      "job_examples": [
        {
          "job_title": "Civil Engineer",
          "work_field_scope": "Designs and oversees the construction of infrastructure projects.",
          "skills": [
            "Bachelor's degree in Civil Engineering",
            "Knowledge of engineering principles and materials",
            "Project management skills"
          ],
          "requirements": [
            "Professional Engineer (PE) license",
            "Relevant experience in civil engineering"
          ]
        },
        {
          "job_title": "Electrical Engineer",
          "work_field_scope": "Designs and tests electrical systems and components.",
          "skills": [
            "Bachelor's degree in Electrical Engineering",
            "Analytical and problem-solving skills",
            "Knowledge of electrical systems and regulations"
          ],
          "requirements": [
            "Professional Engineer (PE) license",
            "Experience in electrical engineering"
          ]
        },
        {
          "job_title": "Software Engineer",
          "work_field_scope": "Develops software applications and systems.",
          "skills": [
            "Proficiency in programming languages",
            "Software development life cycle knowledge",
            "Problem-solving and debugging skills"
          ],
          "requirements": [
            "Bachelor's degree in Computer Science or related field",
            "Experience in software development"
          ]
        }
      ]
    },
    {
      "field_name": "Environmental Science",
      "job_examples": [
        {
          "job_title": "Environmental Scientist",
          "work_field_scope": "Studies and analyzes environmental issues.",
          "skills": [
            "Bachelor's or Master's degree in Environmental Science",
            "Research and analytical skills",
            "Knowledge of environmental regulations"
          ],
          "requirements": [
            "Experience in environmental research",
            "Knowledge of GIS and environmental modeling"
          ]
        },
        {
          "job_title": "Conservation Biologist",
          "work_field_scope": "Works to protect and preserve ecosystems and species.",
          "skills": [
            "Biology or ecology degree",
            "Field research and data collection skills",
            "Knowledge of conservation principles"
          ],
          "requirements": [
            "Master's degree in Conservation Biology or related field",
            "Fieldwork experience in conservation projects"
          ]
        },
        {
          "job_title": "Environmental Engineer",
          "work_field_scope": "Designs solutions to address environmental challenges.",
          "skills": [
            "Bachelor's degree in Environmental Engineering",
            "Problem-solving and design skills",
            "Knowledge of environmental laws and regulations"
          ],
          "requirements": [
            "Professional Engineer (PE) license",
            "Experience in environmental engineering projects"
          ]
        }
      ]
    },
    {
      "field_name": "Human Resources (HR)",
      "job_examples": [
        {
          "job_title": "HR Manager",
          "work_field_scope": "Oversees HR functions, including recruitment, training, and employee relations.",
          "skills": [
            "Leadership and organizational skills",
            "Knowledge of labor laws and regulations",
            "Effective communication and interpersonal skills"
          ],
          "requirements": [
            "Master's degree in HR, Business, or related field",
            "HR certification (e.g., SHRM-SCP) may be required"
          ]
        },
        {
          "job_title": "Recruitment Specialist",
          "work_field_scope": "Sources and recruits candidates for job openings.",
          "skills": [
            "Talent sourcing and interviewing skills",
            "Understanding of recruitment strategies",
            "Communication and negotiation skills"
          ],
          "requirements": [
            "Bachelor's degree in HR, Psychology, or related field",
            "Experience in recruitment"
          ]
        },
        {
          "job_title": "Employee Relations Specialist",
          "work_field_scope": "Handles workplace conflicts and employee relations.",
          "skills": [
            "Conflict resolution and mediation skills",
            "Knowledge of HR policies and procedures",
            "Empathy and interpersonal skills"
          ],
          "requirements": [
            "Bachelor's degree in HR, Psychology, or related field",
            "Experience in employee relations"
          ]
        }
      ]
    }
  ]
}


Overwriting job_data.json


In [ ]:
nltk.download('punkt')
with open('job_data.json') as json_file:
  data = json.load(json_file)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
all_jobs = []

for field in data["job_fields"]:
    for job in field["job_examples"]:
        job_details = {
            "field": field["field_name"],
            "title": job["job_title"],
            "description": job["work_field_scope"],
            "skills": job["skills"]
        }
        all_jobs.append(job_details)


In [ ]:
df = pd.DataFrame(all_jobs)

In [ ]:
df['tokenized_skills'] = df['skills'].apply(lambda skills: word_tokenize(' '.join(skills)) if isinstance(skills, list) else [])
df['tokenized_skills'] = df['tokenized_skills'].apply(lambda tokens: [token.lower() for token in tokens])


In [ ]:
def recommend_jobs(user_field, user_skills):
    # Tokenize user skills
    tokenized_user_skills = word_tokenize(' '.join(user_skills))
    tokenized_user_skills = [token.lower() for token in tokenized_user_skills]

    # Filter jobs based on user's preferred field
    matching_field = df[df['field'] == user_field]

    # Filter jobs based on user's tokenized skills
    matching_skills = matching_field[matching_field['tokenized_skills'].apply(lambda skills: any(skill in skills for skill in tokenized_user_skills))]

    return matching_skills

In [ ]:
user_preferred_field = "Information Technology (IT) and Computer Science"
user_preferred_skills = ["Programming languages", "Problem-solving", "Software development methodologies"]

recommended_jobs = recommend_jobs(user_preferred_field, user_preferred_skills)

# Print recommended jobs
print(recommended_jobs)

                                              field                  title  \
0  Information Technology (IT) and Computer Science     Software Developer   
1  Information Technology (IT) and Computer Science  Network Administrator   
2  Information Technology (IT) and Computer Science         Data Scientist   

                                         description  \
0  Designs, codes, tests, and maintains software ...   
1  Manages and maintains an organization's comput...   
2  Analyzes and interprets complex data sets to i...   

                                              skills  \
0  [Programming languages (e.g., Java, Python, C+...   
1  [Networking protocols and technologies, Troubl...   
2  [Statistics and data analysis skills, Programm...   

                                    tokenized_skills  
0  [programming, languages, (, e.g., ,, java, ,, ...  
1  [networking, protocols, and, technologies, tro...  
2  [statistics, and, data, analysis, skills, prog...  
